## Explore o agrupamento K-Means usando R e os princípios de dados Tidy.

### [**Questionário pré-aula**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

Nesta lição, você aprenderá como criar agrupamentos usando o pacote Tidymodels e outros pacotes do ecossistema R (vamos chamá-los de amigos 🧑‍🤝‍🧑), além do conjunto de dados de música nigeriana que você importou anteriormente. Vamos abordar os fundamentos do K-Means para agrupamento. Lembre-se de que, como você aprendeu na lição anterior, existem várias maneiras de trabalhar com agrupamentos, e o método que você usa depende dos seus dados. Vamos experimentar o K-Means, pois é a técnica de agrupamento mais comum. Vamos começar!

Termos que você aprenderá:

- Pontuação Silhouette

- Método do Cotovelo

- Inércia

- Variância

### **Introdução**

O [agrupamento K-Means](https://wikipedia.org/wiki/K-means_clustering) é um método derivado do domínio do processamento de sinais. Ele é usado para dividir e particionar grupos de dados em `k agrupamentos` com base em semelhanças em suas características.

Os agrupamentos podem ser visualizados como [diagramas de Voronoi](https://wikipedia.org/wiki/Voronoi_diagram), que incluem um ponto (ou 'semente') e sua região correspondente.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infográfico por Jen Looper</figcaption>

O agrupamento K-Means segue os seguintes passos:

1. O cientista de dados começa especificando o número desejado de agrupamentos a serem criados.

2. Em seguida, o algoritmo seleciona aleatoriamente K observações do conjunto de dados para servir como os centros iniciais dos agrupamentos (ou seja, centróides).

3. Depois, cada uma das observações restantes é atribuída ao centróide mais próximo.

4. Em seguida, as novas médias de cada agrupamento são calculadas e o centróide é movido para a nova média.

5. Agora que os centros foram recalculados, cada observação é verificada novamente para ver se pode estar mais próxima de um agrupamento diferente. Todos os objetos são realocados novamente usando as médias atualizadas dos agrupamentos. As etapas de atribuição de agrupamento e atualização dos centróides são repetidas iterativamente até que as atribuições de agrupamento parem de mudar (ou seja, quando a convergência é alcançada). Normalmente, o algoritmo termina quando cada nova iteração resulta em um movimento insignificante dos centróides e os agrupamentos se tornam estáticos.

<div>

> Observe que, devido à randomização das k observações iniciais usadas como centróides iniciais, podemos obter resultados ligeiramente diferentes a cada vez que aplicamos o procedimento. Por essa razão, a maioria dos algoritmos utiliza vários *inícios aleatórios* e escolhe a iteração com o menor WCSS. Assim, é altamente recomendável sempre executar o K-Means com vários valores de *nstart* para evitar um *ótimo local indesejado.*

</div>

Esta breve animação usando a [arte](https://github.com/allisonhorst/stats-illustrations) de Allison Horst explica o processo de agrupamento:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Arte por @allison_horst</figcaption>

Uma questão fundamental que surge no agrupamento é: como saber em quantos agrupamentos separar seus dados? Uma desvantagem do uso do K-Means é o fato de que você precisará estabelecer `k`, ou seja, o número de `centróides`. Felizmente, o `método do cotovelo` ajuda a estimar um bom valor inicial para `k`. Você experimentará isso em breve.

### 

**Pré-requisito**

Vamos continuar de onde paramos na [lição anterior](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), onde analisamos o conjunto de dados, criamos várias visualizações e filtramos o conjunto de dados para observações de interesse. Certifique-se de conferir!

Vamos precisar de alguns pacotes para concluir este módulo. Você pode instalá-los com: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Alternativamente, o script abaixo verifica se você possui os pacotes necessários para completar este módulo e os instala para você caso algum esteja faltando.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Vamos começar com tudo!

## 1. Uma dança com dados: Reduza para os 3 gêneros musicais mais populares

Este é um resumo do que fizemos na lição anterior. Vamos explorar e analisar alguns dados!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 Isso foi ótimo!

## 2. Mais exploração de dados.

Quão limpos estão esses dados? Vamos verificar a presença de outliers usando boxplots. Vamos nos concentrar em colunas numéricas com menos outliers (embora você possa limpar os outliers, se preferir). Boxplots podem mostrar o intervalo dos dados e ajudar a escolher quais colunas usar. Observe que os boxplots não mostram a variância, um elemento importante para dados bem agrupáveis. Por favor, veja [esta discussão](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) para leitura adicional.

[Boxplots](https://en.wikipedia.org/wiki/Box_plot) são usados para representar graficamente a distribuição de dados `numéricos`, então vamos começar *selecionando* todas as colunas numéricas junto com os gêneros musicais populares.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Veja como o seletor `where` facilita isso 💁? Explore outras funções semelhantes [aqui](https://tidyselect.r-lib.org/).

Como vamos criar um boxplot para cada característica numérica e queremos evitar o uso de loops, vamos reformular nossos dados para um formato *mais longo*, o que nos permitirá aproveitar os `facets` - subgráficos que exibem cada um um subconjunto dos dados.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Muito mais longo! Agora é hora de alguns `ggplots`! Então, qual `geom` usaremos?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Fácil-gg!

Agora podemos ver que esses dados estão um pouco ruidosos: ao observar cada coluna como um boxplot, é possível identificar valores atípicos. Você poderia percorrer o conjunto de dados e remover esses valores atípicos, mas isso tornaria os dados bem reduzidos.

Por enquanto, vamos escolher quais colunas usaremos para o nosso exercício de agrupamento. Vamos selecionar as colunas numéricas com intervalos semelhantes. Poderíamos codificar o `artist_top_genre` como numérico, mas vamos descartá-lo por enquanto.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Computando agrupamento k-means em R

Podemos calcular k-means em R usando a função integrada `kmeans`, veja `help("kmeans()")`. A função `kmeans()` aceita um data frame com todas as colunas numéricas como seu argumento principal.

O primeiro passo ao usar o agrupamento k-means é especificar o número de clusters (k) que serão gerados na solução final. Sabemos que há 3 gêneros musicais que extraímos do conjunto de dados, então vamos tentar com 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


O objeto kmeans contém várias informações que estão bem explicadas em `help("kmeans()")`. Por enquanto, vamos nos concentrar em algumas delas. Podemos ver que os dados foram agrupados em 3 clusters com tamanhos de 65, 110 e 111. A saída também contém os centros dos clusters (médias) para os 3 grupos em relação às 5 variáveis.

O vetor de agrupamento é a atribuição de cluster para cada observação. Vamos usar a função `augment` para adicionar a atribuição de cluster ao conjunto de dados original.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Perfeito, acabamos de dividir nosso conjunto de dados em um conjunto de 3 grupos. Então, quão boa é nossa clusterização 🤷? Vamos dar uma olhada no `Silhouette score`.

### **Silhouette score**

[A análise de Silhouette](https://en.wikipedia.org/wiki/Silhouette_(clustering)) pode ser usada para estudar a distância de separação entre os clusters resultantes. Esse score varia de -1 a 1, e se o score estiver próximo de 1, o cluster é denso e bem separado dos outros clusters. Um valor próximo de 0 representa clusters sobrepostos com amostras muito próximas da fronteira de decisão dos clusters vizinhos. [fonte](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

O método de silhouette médio calcula a média do silhouette das observações para diferentes valores de *k*. Um alto score médio de silhouette indica uma boa clusterização.

A função `silhouette` no pacote de cluster é usada para calcular a largura média do silhouette.

> O silhouette pode ser calculado com qualquer [métrica de distância](https://en.wikipedia.org/wiki/Distance "Distance"), como a [distância Euclidiana](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") ou a [distância Manhattan](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance"), que discutimos na [lição anterior](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Nosso índice é **0,549**, bem no meio. Isso indica que nossos dados não são particularmente adequados para este tipo de agrupamento. Vamos verificar se conseguimos confirmar essa suspeita visualmente. O [pacote factoextra](https://rpkgs.datanovia.com/factoextra/index.html) fornece funções (`fviz_cluster()`) para visualizar agrupamentos.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


A sobreposição nos clusters indica que nossos dados não são particularmente adequados para esse tipo de agrupamento, mas vamos continuar.

## 4. Determinando o número ideal de clusters

Uma pergunta fundamental que frequentemente surge no agrupamento K-Means é esta: sem rótulos de classe conhecidos, como você sabe em quantos clusters deve separar seus dados?

Uma maneira de tentar descobrir é usar uma amostra de dados para `criar uma série de modelos de agrupamento` com um número crescente de clusters (por exemplo, de 1 a 10) e avaliar métricas de agrupamento, como o **Silhouette score.**

Vamos determinar o número ideal de clusters calculando o algoritmo de agrupamento para diferentes valores de *k* e avaliando o **Within Cluster Sum of Squares** (WCSS). O total de soma de quadrados dentro dos clusters (WCSS) mede a compacidade do agrupamento, e queremos que ele seja o menor possível, com valores mais baixos indicando que os pontos de dados estão mais próximos.

Vamos explorar o efeito de diferentes escolhas de `k`, de 1 a 10, nesse agrupamento.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Agora que temos o total da soma dos quadrados dentro dos clusters (tot.withinss) para cada algoritmo de agrupamento com centro *k*, usamos o [método do cotovelo](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) para encontrar o número ideal de clusters. O método consiste em plotar o WCSS como uma função do número de clusters e escolher o [cotovelo da curva](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") como o número de clusters a ser utilizado.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


O gráfico mostra uma grande redução no WCSS (ou seja, maior *coerência*) à medida que o número de clusters aumenta de um para dois, e uma redução adicional perceptível de dois para três clusters. Depois disso, a redução é menos acentuada, resultando em um `cotovelo` 💪 no gráfico em torno de três clusters. Isso é uma boa indicação de que há dois a três clusters de pontos de dados razoavelmente bem separados.

Agora podemos prosseguir e extrair o modelo de clustering onde `k = 3`:

> `pull()`: usado para extrair uma única coluna
>
> `pluck()`: usado para indexar estruturas de dados como listas


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Ótimo! Vamos visualizar os clusters obtidos. Que tal adicionar um pouco de interatividade usando `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Talvez esperássemos que cada cluster (representado por cores diferentes) tivesse gêneros distintos (representados por formas diferentes).

Vamos dar uma olhada na precisão do modelo.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


A precisão deste modelo não é ruim, mas também não é excelente. Pode ser que os dados não sejam adequados para o agrupamento K-Means. Esses dados são muito desequilibrados, pouco correlacionados e há muita variância entre os valores das colunas para formar bons clusters. Na verdade, os clusters que se formam provavelmente são fortemente influenciados ou distorcidos pelas três categorias de gênero que definimos acima.

Mesmo assim, foi um processo de aprendizado interessante!

Na documentação do Scikit-learn, você pode ver que um modelo como este, com clusters não muito bem definidos, apresenta um problema de 'variância':

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infográfico do Scikit-learn</figcaption>



## **Variância**

Variância é definida como "a média das diferenças quadradas em relação à média" [fonte](https://www.mathsisfun.com/data/standard-deviation.html). No contexto deste problema de agrupamento, refere-se aos dados cujos números do nosso conjunto tendem a divergir um pouco demais da média.

✅ Este é um ótimo momento para pensar em todas as maneiras de corrigir esse problema. Ajustar os dados um pouco mais? Usar colunas diferentes? Utilizar um algoritmo diferente? Dica: Experimente [escalar seus dados](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) para normalizá-los e testar outras colunas.

> Experimente este '[calculador de variância](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' para entender melhor o conceito.

------------------------------------------------------------------------

## **🚀Desafio**

Dedique algum tempo a este notebook, ajustando os parâmetros. Você consegue melhorar a precisão do modelo limpando mais os dados (removendo outliers, por exemplo)? Você pode usar pesos para dar mais importância a determinadas amostras de dados. O que mais você pode fazer para criar clusters melhores?

Dica: Experimente escalar seus dados. Há código comentado no notebook que adiciona escalonamento padrão para fazer com que as colunas de dados se assemelhem mais em termos de intervalo. Você verá que, embora a pontuação de silhueta diminua, o 'cotovelo' no gráfico suaviza. Isso ocorre porque deixar os dados sem escala permite que dados com menos variância tenham mais peso. Leia mais sobre esse problema [aqui](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Quiz pós-aula**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Revisão e Autoestudo**

-   Dê uma olhada em um simulador de K-Means [como este](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Você pode usar esta ferramenta para visualizar pontos de dados de amostra e determinar seus centróides. Você pode editar a aleatoriedade dos dados, o número de clusters e o número de centróides. Isso ajuda você a ter uma ideia de como os dados podem ser agrupados?

-   Além disso, confira [este material sobre K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) de Stanford.

Quer testar suas habilidades recém-adquiridas de agrupamento em conjuntos de dados que se adaptam bem ao K-Means? Veja:

-   [Treine e avalie modelos de agrupamento](https://rpubs.com/eR_ic/clustering) usando Tidymodels e amigos

-   [Análise de Cluster K-Means](https://uc-r.github.io/kmeans_clustering), Guia de Programação R para Análise de Negócios da UC

- [Agrupamento K-Means com princípios de dados organizados](https://www.tidymodels.org/learn/statistics/k-means/)

## **Tarefa**

[Experimente diferentes métodos de agrupamento](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## AGRADECIMENTOS A:

[Jen Looper](https://www.twitter.com/jenlooper) por criar a versão original em Python deste módulo ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) por criar as ilustrações incríveis que tornam o R mais acolhedor e envolvente. Encontre mais ilustrações em sua [galeria](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Feliz aprendizado,

[Eric](https://twitter.com/ericntay), Embaixador Estudante Gold da Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Arte por @allison_horst</figcaption>



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte oficial. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações equivocadas decorrentes do uso desta tradução.
